# Modèle linéaire

Considérons la cas classique d'une fonction affine :

$$y=ax+b$$

Ici, $a$ et $b$ sont des réels. Ces deux nombres définissent entièrement la courbe et permet donc d'obtenir une relation **affine** entre $x$ et $y$. En statistique, cette relation est à la base des modèles dit **linéaires**, où une variable réponse se définit comme une somme de variables explicatives où chacune de ces dernières sont multipliés par un coefficient.


## Modèle linéaire simple

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/438px-Linear_regression.svg.png)

Dans le modèle linéaire simple (une seule variable explicative), on suppose que la variable réponse suit le modèle suivant :

$$y_i=\beta_0 + \beta_1 x_i + \varepsilon_i$$

On remarque la ressemblance avec la fonction affine présentée ci-dessus. La différence réside dans l'existence du terme aléatoire (appelé bruit) $\varepsilon_i$. Afin de considérer le modèle, il est nécessaire de se placer sous les hypothèses suivantes

$$(\mathcal{H}) : \left\{\begin{matrix}
\mathbb{E}[\varepsilon_i]=0\\ 
\text{Cov}(\varepsilon_i, \varepsilon_j)=\delta_{ij} \sigma^2
\end{matrix}\right.$$
Les différents éléments qui interviennent sont :

- $\beta_0$ : l'ordonnée à l'origine (nommée *intercept*)
- $\beta_1$ : le coefficient directeur
- $x_i$ : l'observation $i$
- $y_i$ : le $i$-ème prix
- $\varepsilon_i$ : le bruit aléatoire liée à la $i$-ème observation

La solution peut se calculer facilement via les formules fermées suivantes :

$$\hat{\beta}_1=\frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2} \qquad \hat{\beta}_0 = \hat{y} - \hat{\beta}_1 \bar{x}$$

## Modèle linéaire multiple

Dans le cas multiple (pour $p$ variables explicatives), pour la $i$-ème observation, le modèle s'écrit :

$$y_i= \beta_0 + \sum_{j=1}^p \beta_j x_{ij} + \varepsilon_i$$

Ainsi, une observation $x_i$ n'est plus une valeur, mais un **vecteur** $(x_{i1}, \dots, x_{ip})$. Il est plus commode de regrouper ces prix $y_i$ et ces vecteurs d'observations $x_i$ dans des matrices :

$$Y=X \beta + \varepsilon$$

Sous les hypothèses équivalentes du modèle simple en plus grand dimension

$$(\mathcal{H}) : \left\{\begin{matrix}
\text{rank}(X)=p\\ 
\mathbb{E}[\varepsilon]=0 \text{ et }\text{Var}(\varepsilon)=\sigma^2 I_p
\end{matrix}\right.$$

Les différents éléments qui interviennent sont :

- $\beta$ : le vecteur directeur
- $X$ : la matrice des observations
- $Y$ : le vecteur de prix
- $\varepsilon$ : le vecteur de bruit

Avec $X=( \mathbf{1}, X_1, \dots, X_n)$, $Y=(y_1, \dots, y_n)^\top$ et $\varepsilon=(\varepsilon_1, \dots, \varepsilon_n)^\top$. La solution des MCO (Moindres Carrés Ordinaires) est alors :

$$\hat{\beta}= (X^\top X)^{-1} X^\top Y$$

Vous pouvez d'ailleurs faire la démonstration de votre coté ! Pour plus d'information mathématiques, le portail de wikipédia qui est très bien fait : [lien ici](https://fr.wikipedia.org/wiki/Portail:Probabilit%C3%A9s_et_statistiques)

# Implémenter une régression linéaire 


In [1]:
#importer vos librairies 
import pandas as pd
import numpy as np
import sys, os, json
import scipy
import matplotlib
import sklearn

In [2]:
#charger les données 
#price_availability.csv
#listings_final.csv
price_availability = pd.read_csv('data/price_availability.csv', sep=';')
listings_final = pd.read_csv('data/listings_final.csv', sep=';')

#vérifier si tous les individus ont bien un prix 

price_availability.head()

listing_id         day                       created  available  \
0     9810829  2018-12-08  2018-09-27 06:14:10.000+0000       True   
1     9810829  2018-12-08  2018-09-26 19:34:02.000+0000       True   
2    20897010  2018-12-09  2018-09-27 10:38:57.000+0000       True   
3    20897010  2018-12-09  2018-09-27 06:10:27.000+0000       True   
4    20897010  2018-12-09  2018-09-26 19:30:25.000+0000       True   

  local_currency  local_price  min_nights  
0            EUR          160           1  
1            EUR          160           1  
2            EUR          172           2  
3            EUR          172           2  
4            EUR          172           2

In [3]:
listings_final.head()

Unnamed: 0  listing_id                                      name  \
0           0    28581061                           La maison Clery   
1           1      661961       studio PARIS PLACE EDITH PIAF 75020   
2           2     1261705  chambre privée à louer @ paris oberkampf   
3           3     1318834            Appartement au coeur du Marais   
4           4     1677091                       Lovely & Quiet flat   

           type   city                  neighborhood   latitude  longitude  \
0  private_room  Paris             2e arrondissement  48.869292   2.348335   
1   entire_home  Paris                           NaN  48.867284   2.403255   
2  private_room  Paris                           NaN  48.867894   2.375897   
3   entire_home  Paris                    République  48.870370   2.358510   
4   entire_home  Paris  Buttes-Chaumont - Belleville  48.874149   2.373700   

   person_capacity  beds  bedrooms  bathrooms  is_rebookable  is_new_listing  \
0                1     1         1        2.0          False            True   
1                2     1         1        1.0          False           False   
2                1     1         1        1.0          False           False   
3                3     2         2        1.0          False           False   
4                2     1         1        1.0          False           False   

   is_fully_refundable  is_host_highly_rated  is_business_travel_ready  \
0                 True                 False                     False   
1                 True                  True                     False   
2                 True                  True                     False   
3                 True                 False                     False   
4                 True                  True                     False   

   pricing_weekly_factor  pricing_monthly_factor  
0                   1.00                    1.00  
1                   0.88                    0.69  
2                   1.00                    1.00  
3                   0.82                    0.48  
4                   0.95                    0.90

## Données d'entrée

L'objectif ici est de charger les données pour créer les matrices $X$ et $Y$ du modèle linéaire. **Attention**, il n'est pas nécessaire de rajouter le vecteur colonne $\mathbf{1}$ en première colonne, car *scikit-learn* le fait automatiquement !

In [4]:
#définir 2 variables de travail
#X := les features à utiliser 
#Y := la target (prix)
x = 'local_price'

price_group = price_availability.groupby(['listing_id']).mean()
price_group

available  local_price  min_nights
listing_id                                    
5396         0.011494   102.363985    2.000000
7397         0.168297   110.107632   11.000000
9342         0.309198   396.863014    2.784736
10010        0.254593   136.154856    4.955381
10270        0.511688   105.779221    4.937662
...               ...          ...         ...
28836096     0.082707   321.428571    1.022556
28838519     0.083333    54.483333    5.000000
28840013     0.743083   112.122530    2.000000
28846494     0.225564   168.037594    4.000000
28851976     0.812030   128.000000    2.000000

[11749 rows x 3 columns]

In [5]:
merge = pd.merge(listings_final, price_group, on=['listing_id'])
merge

Unnamed: 0  listing_id                                      name  \
0             0    28581061                           La maison Clery   
1             1      661961       studio PARIS PLACE EDITH PIAF 75020   
2             2     1261705  chambre privée à louer @ paris oberkampf   
3             3     1318834            Appartement au coeur du Marais   
4             4     1677091                       Lovely & Quiet flat   
..          ...         ...                                       ...   
994         995    28335197          Studio cosy Jardin du Luxembourg   
995         996    28583013    Charmant  30m2 - Faubourg Saint Martin   
996         997    28628316       Cosy flat in the marais - Best area   
997         998    28792796         Appartement 3 chambres madeleine.   
998         999    28682896                          Village Jourdain   

             type   city                    neighborhood   latitude  \
0    private_room  Paris               2e arrondissement  48.869292   
1     entire_home  Paris                             NaN  48.867284   
2    private_room  Paris                             NaN  48.867894   
3     entire_home  Paris                      République  48.870370   
4     entire_home  Paris    Buttes-Chaumont - Belleville  48.874149   
..            ...    ...                             ...        ...   
994   entire_home  Paris  Saint Germain des Prés - Odéon  48.848695   
995   entire_home  Paris                      République  48.871623   
996   entire_home  Paris               2e arrondissement  48.867434   
997   entire_home  Paris             Madeleine - Vendôme  48.870109   
998  private_room  Paris    Buttes-Chaumont - Belleville  48.875425   

     longitude  person_capacity  beds  ...  is_rebookable  is_new_listing  \
0     2.348335                1     1  ...          False            True   
1     2.403255                2     1  ...          False           False   
2     2.375897                1     1  ...          False           False   
3     2.358510                3     2  ...          False           False   
4     2.373700                2     1  ...          False           False   
..         ...              ...   ...  ...            ...             ...   
994   2.325857                2     1  ...          False            True   
995   2.358006                3     1  ...          False            True   
996   2.351771                4     2  ...          False            True   
997   2.321475                6     4  ...          False            True   
998   2.395240                2     1  ...          False            True   

     is_fully_refundable  is_host_highly_rated  is_business_travel_ready  \
0                   True                 False                     False   
1                   True                  True                     False   
2                   True                  True                     False   
3                   True                 False                     False   
4                   True                  True                     False   
..                   ...                   ...                       ...   
994                 True                 False                     False   
995                 True                 False                     False   
996                 True                  True                     False   
997                 True                 False                     False   
998                 True                 False                     False   

     pricing_weekly_factor  pricing_monthly_factor  available  local_price  \
0                     1.00                    1.00   0.234536    45.948454   
1                     0.88                    0.69   0.030075    44.473684   
2                     1.00                    1.00   0.744624    49.000000   
3                     0.82                    0.48   0.953125   165.000000   
4                     0.95                    0

In [6]:
x = merge[['person_capacity','bathrooms','bedrooms']]
x

person_capacity  bathrooms  bedrooms
0                  1        2.0         1
1                  2        1.0         1
2                  1        1.0         1
3                  3        1.0         2
4                  2        1.0         1
..               ...        ...       ...
994                2        1.0         0
995                3        1.0         1
996                4        1.0         1
997                6        1.5         2
998                2        1.0         1

[999 rows x 3 columns]

In [7]:
y = merge[['local_price']]
y

local_price
0      45.948454
1      44.473684
2      49.000000
3     165.000000
4      69.774436
..           ...
994    38.507732
995   147.752551
996   152.860825
997    49.184211
998    39.503937

[999 rows x 1 columns]

In [8]:
#construire l'ensemble de donnée prix 
#
#    INDICE 
# 
# récupérer les prix des ID dans le dataset de prix 
# 🚧 il y a plusieurs prix dans le dataset 🚧



En *Machine Learning*, on a l'habitude de couper l'ensemble de données en deux sous-ensembles :

- Un ensemble d'entraînement (*train set*), sur lequel le modèle va être calibré.
- Un ensemble de test (*test set*), qui ne sera pas utilisé pendant le calibrage mais permettra de vérifier l'aptitude du modèle à généraliser sur de nouvelles observations inconnues.

En général, on découpe l'ensemble de données (*split*) en prenant $\alpha \%$ de l'ensemble pour entraînement et $1-\alpha \%$ comme test. Dans la plus part des cas, on considère que $\alpha=10,20 ou 30\%$.

In [9]:
#utiliser la méthode split de sklearn en splitant avec un alpha=30 et un random state=42 
#zafficher la shape de vos données 

from sklearn.model_selection import train_test_split

#On prend 30% (O.3) d'entrainement et 7O% de test
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

In [10]:
x_train.shape

(699, 3)

In [11]:
y_train.shape

(699, 1)

Pour information, *scikit-learn* utilise le solveur OLS (Ordinary Least Squares) de *numpy*.

In [16]:
#créer l'objet de régression et entrainer le sur notre ensemble d'entraînement
from sklearn import linear_model

reg = linear_model.LinearRegression().fit(x_train,y_train)

coef beta_j : [[33.07789571 83.12135251  8.545884  ]]
coef intercept beta_0 : [-42.85095177]


On affiche le vecteur des coefficients pour interpréter rapidement le modèle.

In [13]:
#afficher les coefficients
#que remarquez vous ? 
print('coef beta_j : {}'.format(reg.coef_))
print('coef intercept beta_0 : {}'.format(reg.intercept_))

#Calcul trouvé = (person_capacity*33,O78 + bathrooms*83,12 + bedrooms*8,54) - 42,85
#... pas ouf ...

## Validation du modèle

### Le coefficient de détermination $R^2$

Par la suite, nous ferons l'hypothèse de gaussianité sur les bruits. Dans l'idée, nous aimerions obtenir une valeur numérique qui nous indique à quel point la régression linéaire a un sens sur nos données. Pour cela, introduisons les notations suivantes :

- $SCT=\|Y-\hat{y} \mathbf{1}\|^2$ est la somme des carrés totaux
- $SCE=\|\hat{Y}-\hat{y} \mathbf{1}\|^2$ est la somme des carrés expliqués
- $SCR=\|\hat{\varepsilon}\|^2$ est la somme des carrés résiduels

L'idée est de décomposer la somme des carrés totaux comme la somme des carrés que le modèle explique, en plus de la somme des carrés qui sont liés aux résidus (et donc que le modèle ne peut pas expliquer). On voit donc ici l'intérêt de calculer un coefficient à partir du $SCE$. Puisque l'on a la relation suivante :

$$SCT=SCE+SCR \text{ alors } 1=\frac{SCE}{SCT}+\frac{SCR}{SCT}$$

Plus les résidus sont petits (et donc la régression est "bonne"), plus $SCR$ devient petit et donc $SCE$ devient grand. Le schéma inverse s'opère de la même façon. Dans le meilleur des cas, on obtient $SCR=0$ et donc $SCE=SCT$ d'où le premier membre vaut $1$. Dans le cas contraite, $SCE=0$ et automatiquement, le premier membre est nul. C'est ainsi que l'on définit le coefficient de détermination $R^2$ comme 
$$R^2=\frac{SCE}{SCT}=1-\frac{SCR}{SCT}$$
Ainsi, $R^2 \in [0,1]$. Plus $R^2$ est proche de $1$, plus la régression linéaire a du sens. Au contraire, si $R^2$ est proche de $0$, le modèle linéaire possède un faible pouvoir explicatif.

In [23]:
#faire une prediction sur X
y_pred = reg.predict(x_train)

In [33]:
#afficher l'erreur des moindres carrées sur l'ensemble d'entrainement ainsi que le R2
from sklearn.metrics import r2_score, mean_squared_error

print("Erreur des moindres carrés : {}".format(mean_squared_error(y_train, y_pred)))
print("Score R2 : {}".format(r2_score(y_train, y_pred)))

Erreur des moindres carrés : 22564.83450544439
Score R2 : 0.3363475067727145


Erreur des moindres carrés : 22564.83450544439


## Bonus : Analyse de l'homoscédasticité

L'analyse de l'homoscédasticité est primordiale : c'est en particulier elle qui nous permet de vérifier, à partir des résidus, si les bruits vérifient bien l'hypothèse $(\mathcal{H})$. On calcule donc les **résidus studentisés**.

$$t_i^*=\frac{\hat{\varepsilon}_i}{\hat{\sigma}_{(i)} \sqrt{1-h_{ii}}}$$
Avec $h_{ii}=\{X(X^\top X)^{-1} X^\top\}_{ii}=H_{ii}$ la matrice de projection sur l'hyperplan des variables. Plus précisément, $H$ est la matrice qui projette $Y$ sur l'espace engendré par les variables, soit $\hat{Y}=HY$. De même, on considère $\hat{\sigma}_{(i)}$ l'estimateur de la variance du bruit en supprimant l'observation $i$ (par une méthode de validation croisée Leave-One-Out que nous ne détaillerons pas ici).

Dans ce cas, on peut montrer que les résidus studentisés suivent une loi de Student à $n-p-1$ degrés de liberté.

In [15]:
#analyser le code ci-dessous 
import scipy
Y_pred = reg.predict(x_train)
n = x_train.shape[0]
p = 4
residuals = np.abs(y_train - Y_pred)
H = np.matmul(x_train, np.linalg.solve(np.dot(x_train.T, x_train), x_train.T))
std_hat = np.dot(residuals, residuals) / (n - p)
standart_residuals = np.asarray([residuals[i] / np.sqrt(std_hat * (1 - H[i, i])) for i in range(len(residuals))])
student_residuals = np.asarray([ standart_residuals[i] * np.sqrt((n - p - 1) / (n - p - standart_residuals[i]**2)) for i in range(n) ])
cook = np.asarray([ H[i, i] * student_residuals[i] / (x_train.shape[1] * (1 - H[i, i])) for i in range(n) ])

plt.figure(figsize=(20, 12))
plt.subplot(221)
plt.scatter(Y_pred, student_residuals, s=12, c="white", edgecolors="blue")
plt.plot([min(Y_pred), max(Y_pred)], [ scipy.stats.t.ppf(q=0.975, df=n-p-1), scipy.stats.t.ppf(q=0.975, df=n-p-1)], color="green", alpha=0.6, label="Quantile de Student")
plt.title("Analyse de l’homoscédasticité")
plt.xlabel("Prédictions $\hat{y}_i$")
plt.ylabel("Résidus studentisés $|t_i^*|$")
plt.legend()

ValueError: shapes (699,1) and (699,1) not aligned: 1 (dim 1) != 699 (dim 0)